In [1]:
import os
import sys
import argparse
import tensorflow as tf
import numpy as np


image_height = 128 
image_width = 128 
real_width = 394
real_height = 394
real_depth = 234
condition_shape = 7

total_epoch = 1
batch_size = 1
shuffle_buffer_size = 1

test_input_dir = "D:/new202103/test_input/test1_64" 
test_output_dir = "D:/new202103/test_output/test1_64"
test_condition_dir = "D:/new202103/movement_grad2/test1/9"
result_dir =  './output/result/5/gradient/test1' 
ckpt_dir = './output/checkpoint/5' 

In [2]:
EPS = 1e-12





# Class for batch normalization node

class batch_norm(object):

    def __init__(self, epsilon=1e-5, momentum=0.9, name="batch_norm"):

        with tf.variable_scope(name):

            self.epsilon = epsilon

            self.momentum = momentum

            self.name = name



    def __call__(self, x, train=True):

        return tf.contrib.layers.batch_norm(x,

                                            decay=self.momentum,

                                            updates_collections=None,

                                            epsilon=self.epsilon,

                                            scale=True,

                                            is_training=train,

                                            scope=self.name,

                                            reuse=tf.AUTO_REUSE     # if tensorflow vesrion < 1.4, delete this line

                                            )





# leaky relu function

def lrelu(X, leak=0.2):

    f1 = 0.5 * (1 + leak)

    f2 = 0.5 * (1 - leak)

    return f1 * X + f2 * tf.abs(X)





class Pix2Pix:

        
    def generate(self, input_img, con_input):
        dummy = np.ones((1,64,64,1))
        img_concat = tf.concat([con_input[0,0,0,0]*dummy, con_input[0,0,0,1]*dummy,con_input[0,0,0,2]*dummy,con_input[0,0,0,3]*dummy,con_input[0,0,0,4]*dummy,con_input[0,0,0,5]*dummy,con_input[0,0,0,6]*dummy], axis=3)
        
        
        h1 = h1_ = tf.nn.conv2d(input_img, self.G_W1, strides=[1, 2, 2, 1], padding='SAME')  

        h1 = lrelu(h1)

        h1 = tf.concat([h1, img_concat], axis=3)


        h2 = tf.nn.conv2d(h1, self.G_W2, strides=[1, 2, 2, 1], padding='SAME')  

        h2 = h2_ = self.G_bn2(h2)

        h2 = lrelu(h2)
            



        h3 = tf.nn.conv2d(h2, self.G_W3, strides=[1, 2, 2, 1], padding='SAME') 

        h3 = h3_ = self.G_bn3(h3)

        h3 = lrelu(h3)



        h4 = tf.nn.conv2d(h3, self.G_W4, strides=[1, 2, 2, 1], padding='SAME')  

        h4 = h4_ = self.G_bn4(h4)

        h4 = lrelu(h4)



        h5 = tf.nn.conv2d(h4, self.G_W5, strides=[1, 2, 2, 1], padding='SAME')  

        h5 = h5_ = self.G_bn5(h5)

        h5 = lrelu(h5)



        h6 = tf.nn.conv2d(h5, self.G_W6, strides=[1, 2, 2, 1], padding='SAME') 

        h6 = h6_ = self.G_bn6(h6)

        h6 = lrelu(h6)



        h7 = tf.nn.conv2d(h6, self.G_W7, strides=[1, 2, 2, 1], padding='SAME')  

        h7 = h7_ = self.G_bn7(h7)

        h7 = lrelu(h7)
        
        



        h9 = tf.nn.conv2d_transpose(h7, self.G_W9, output_shape=[batch_size, 2, 2, self.ch_G9], strides=[1, 2, 2, 1])  

        h9 = tf.nn.dropout(self.G_bn9(h9), keep_prob=self.keep_prob)

        h9 = tf.nn.relu(h9)

        h9 = tf.concat([h9, h6_], axis=3)



        h10 = tf.nn.conv2d_transpose(h9, self.G_W10, output_shape=[batch_size, 4, 4, self.ch_G10], strides=[1, 2, 2, 1])  

        h10 = tf.nn.dropout(self.G_bn10(h10), keep_prob=self.keep_prob)

        h10 = tf.nn.relu(h10)

        h10 = tf.concat([h10, h5_], axis=3)



        h11 = tf.nn.conv2d_transpose(h10, self.G_W11, output_shape=[batch_size, 8, 8, self.ch_G11], strides=[1, 2, 2, 1])  

        h11 = tf.nn.dropout(self.G_bn11(h11), keep_prob=self.keep_prob)

        h11 = tf.nn.relu(h11)

        h11 = tf.concat([h11, h4_], axis=3)



        h12 = tf.nn.conv2d_transpose(h11, self.G_W12, output_shape=[batch_size, 16, 16, self.ch_G12], strides=[1, 2, 2, 1]) 

        h12 = self.G_bn12(h12)

        h12 = tf.nn.relu(h12)

        h12 = tf.concat([h12, h3_], axis=3)



        h13 = tf.nn.conv2d_transpose(h12, self.G_W13, output_shape=[batch_size, 32, 32, self.ch_G13], strides=[1, 2, 2, 1])  
        
        h13 = self.G_bn13(h13)

        h13 = tf.nn.relu(h13)

        h13 = tf.concat([h13, h2_], axis=3)



        h14 = tf.nn.conv2d_transpose(h13, self.G_W14, output_shape=[batch_size, 64, 64, self.ch_G14], strides=[1, 2, 2, 1])  

        h14 = self.G_bn14(h14)

        h14 = tf.nn.relu(h14)

        h14 = tf.concat([h14, h1_], axis=3)




        h16 = tf.nn.conv2d_transpose(h14, self.G_W16, output_shape=[batch_size, 128, 128, self.ch_G16], strides=[1, 2, 2, 1])  

        h16 = tf.nn.tanh(h16)


        return h16

    
    
    # Network Parameters

    def __init__(self, sess, batch_size):

        self.learning_rate = 0.00001  



        self.sess = sess

        self.batch_size = batch_size

        self.keep_prob = 0.5

        self.input_image_shape = [image_height, image_width, 1] 
        self.input_condition_shape = [1,1,condition_shape] 
        self.output_image_shape = [image_height, image_width, 1]

        self.l1_weight = 1000.0 



        '''channels'''

        # Gen_Encoding

        self.ch_G0 = 1

        self.ch_G1 = 64-condition_shape 

        self.ch_G2 = 128

        self.ch_G3 = 256

        self.ch_G4 = 512

        self.ch_G5 = 512

        self.ch_G6 = 512

        self.ch_G7 = 512
        
        self.ch_condition = condition_shape


        # Gen_Decoding

        self.ch_G9 = 512

        self.ch_G10 = 512

        self.ch_G11 = 512

        self.ch_G12 = 256

        self.ch_G13 = 128
        
        self.ch_G14 = 64

        self.ch_G16 = 1

        # Discrim

        self.ch_D0 = 2+condition_shape 

        self.ch_D1 = 64

        self.ch_D2 = 128

        self.ch_D3 = 256

        self.ch_D4 = 512

        self.ch_D5 = 1



        '''parameters'''

        # Gen_encoding

        self.G_W1 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G0, self.ch_G1], stddev=0.02), name="G_W1")



        self.G_W2 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G1+7, self.ch_G2], stddev=0.02), name='G_W2')

        self.G_bn2 = batch_norm(name="G_bn2")



        self.G_W3 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G2, self.ch_G3], stddev=0.02), name='G_W3')

        self.G_bn3 = batch_norm(name="G_bn3")



        self.G_W4 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G3, self.ch_G4], stddev=0.02), name='G_W4')

        self.G_bn4 = batch_norm(name="G_bn4")



        self.G_W5 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G4, self.ch_G5], stddev=0.02), name='G_W5')

        self.G_bn5 = batch_norm(name="G_bn5")



        self.G_W6 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G5, self.ch_G6], stddev=0.02), name='G_W6')

        self.G_bn6 = batch_norm(name="G_bn6")



        self.G_W7 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G6, self.ch_G7], stddev=0.02), name='G_W7')

        self.G_bn7 = batch_norm(name="G_bn7")



        # Gen_Decoding

        self.G_W9 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G9, self.ch_G7], stddev=0.02), name='G_W9')

        self.G_bn9 = batch_norm(name="G_bn9")



        self.G_W10 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G10, self.ch_G9 + self.ch_G6], stddev=0.02), name='G_W10')

        self.G_bn10 = batch_norm(name="G_bn10")



        self.G_W11 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G11, self.ch_G10 + self.ch_G5], stddev=0.02), name='G_W11')

        self.G_bn11 = batch_norm(name="G_bn11")



        self.G_W12 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G12, self.ch_G11 + self.ch_G4], stddev=0.02), name='G_W12')

        self.G_bn12 = batch_norm(name="G_bn12")



        self.G_W13 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G13, self.ch_G12 + self.ch_G3], stddev=0.02), name='G_W13')

        self.G_bn13 = batch_norm(name="G_bn13")



        self.G_W14 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G14, self.ch_G13 + self.ch_G2], stddev=0.02), name='G_W14')

        self.G_bn14 = batch_norm(name="G_bn14")




        self.G_W16 = tf.Variable(tf.truncated_normal([4, 4, self.ch_G16, self.ch_G14 + self.ch_G1], stddev=0.02), name='G_W16')



        # Discrim

        self.D_W1 = tf.Variable(tf.truncated_normal([4, 4, self.ch_D0, self.ch_D1], stddev=0.02), name='D_W1')

        self.D_bn1 = batch_norm(name="D_bn1")



        self.D_W2 = tf.Variable(tf.truncated_normal([4, 4, self.ch_D1, self.ch_D2], stddev=0.02), name='D_W2')

        self.D_bn2 = batch_norm(name="D_bn2")



        self.D_W3 = tf.Variable(tf.truncated_normal([4, 4, self.ch_D2, self.ch_D3], stddev=0.02), name='D_W3')

        self.D_bn3 = batch_norm(name="D_bn3")



        self.D_W4 = tf.Variable(tf.truncated_normal([4, 4, self.ch_D3, self.ch_D4], stddev=0.02), name='D_W4')

        self.D_bn4 = batch_norm(name="D_bn4")



        self.D_W5 = tf.Variable(tf.truncated_normal([4, 4, self.ch_D4, self.ch_D5], stddev=0.02), name='D_W5')



        self.gen_params = [

            self.G_W1,

            self.G_W2,

            self.G_W3,

            self.G_W4,

            self.G_W5,

            self.G_W6,

            self.G_W7,

            self.G_W9,

            self.G_W10,

            self.G_W11,

            self.G_W12,

            self.G_W13,

            self.G_W14,

            self.G_W16

        ]



        self.discrim_params = [

            self.D_W1,

            self.D_W2,

            self.D_W3,

            self.D_W4,

            self.D_W5

        ]



        self._build_model()


    def discriminate(self, input_img, target, condition):
        

        dummy = np.ones((1,image_height,image_width,1))
        if condition_shape==7:
            img_concat = tf.concat([input_img, target, condition[0,0,0,0]*dummy, condition[0,0,0,1]*dummy,condition[0,0,0,2]*dummy,condition[0,0,0,3]*dummy,condition[0,0,0,4]*dummy,condition[0,0,0,5]*dummy,condition[0,0,0,6]*dummy], axis=3)
        if condition_shape==6:
            img_concat = tf.concat([input_img, target, condition[0,0,0,0]*dummy, condition[0,0,0,1]*dummy,condition[0,0,0,2]*dummy,condition[0,0,0,3]*dummy,condition[0,0,0,4]*dummy,condition[0,0,0,5]*dummy], axis=3)
        if condition_shape==4:
            img_concat = tf.concat([input_img, target, condition[0,0,0,0]*dummy, condition[0,0,0,1]*dummy,condition[0,0,0,2]*dummy,condition[0,0,0,3]*dummy], axis=3)

       


        h1 = tf.nn.conv2d(img_concat, self.D_W1, strides=[1, 2, 2, 1], padding='SAME')  

        h1 = self.D_bn1(h1)

        h1 = lrelu(h1)



        h2 = tf.nn.conv2d(h1, self.D_W2, strides=[1, 2, 2, 1], padding='SAME')  

        h2 = self.D_bn2(h2)

        h2 = lrelu(h2)



        h3 = tf.nn.conv2d(h2, self.D_W3, strides=[1, 2, 2, 1], padding='SAME')  

        h3 = self.D_bn3(h3)

        h3 = lrelu(h3)



        h4 = tf.pad(h3, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='CONSTANT')  

        h4 = tf.nn.conv2d(h4, self.D_W4, strides=[1, 1, 1, 1], padding='VALID')

        h4 = self.D_bn4(h4)

        h4 = lrelu(h4)



        h5 = tf.pad(h4, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='CONSTANT')  

        h5 = tf.nn.conv2d(h5, self.D_W5, strides=[1, 1, 1, 1], padding='VALID')

        h5 = tf.nn.sigmoid(h5)



        return h5



    # Method for generating the fake images

    def sample_generator(self, input_image, cond_input, batch_size=1):

        input_img = tf.placeholder(tf.float32, [batch_size] + self.input_image_shape)
        con_input = tf.placeholder(tf.float32, [batch_size] + self.input_condition_shape)
        
        dummy = np.ones((1,64,64,1))
        img_concat = tf.concat([con_input[0,0,0,0]*dummy, con_input[0,0,0,1]*dummy,con_input[0,0,0,2]*dummy,con_input[0,0,0,3]*dummy,con_input[0,0,0,4]*dummy,con_input[0,0,0,5]*dummy,con_input[0,0,0,6]*dummy], axis=3)
        
        
        h1 = h1_ = tf.nn.conv2d(input_img, self.G_W1, strides=[1, 2, 2, 1], padding='SAME')  

        h1 = lrelu(h1)

        h1 = tf.concat([h1, img_concat], axis=3)



        h2 = tf.nn.conv2d(h1, self.G_W2, strides=[1, 2, 2, 1], padding='SAME')  

        h2 = h2_ = self.G_bn2(h2)

        h2 = lrelu(h2)



        h3 = tf.nn.conv2d(h2, self.G_W3, strides=[1, 2, 2, 1], padding='SAME')  

        h3 = h3_ = self.G_bn3(h3)

        h3 = lrelu(h3)



        h4 = tf.nn.conv2d(h3, self.G_W4, strides=[1, 2, 2, 1], padding='SAME')  

        h4 = h4_ = self.G_bn4(h4)

        h4 = lrelu(h4)



        h5 = tf.nn.conv2d(h4, self.G_W5, strides=[1, 2, 2, 1], padding='SAME')  

        h5 = h5_ = self.G_bn5(h5)

        h5 = lrelu(h5)



        h6 = tf.nn.conv2d(h5, self.G_W6, strides=[1, 2, 2, 1], padding='SAME')  

        h6 = h6_ = self.G_bn6(h6)

        h6 = lrelu(h6)



        h7 = tf.nn.conv2d(h6, self.G_W7, strides=[1, 2, 2, 1], padding='SAME')  

        h7 = h7_ = self.G_bn7(h7)

        h7 = lrelu(h7)
        
      



        h9 = tf.nn.conv2d_transpose(h7, self.G_W9, output_shape=[batch_size, 2, 2, self.ch_G9], strides=[1, 2, 2, 1])  

        h9 = tf.nn.dropout(self.G_bn9(h9), keep_prob=self.keep_prob)

        h9 = tf.nn.relu(h9)

        h9 = tf.concat([h9, h6_], axis=3)



        h10 = tf.nn.conv2d_transpose(h9, self.G_W10, output_shape=[batch_size, 4, 4, self.ch_G10], strides=[1, 2, 2, 1])  

        h10 = tf.nn.dropout(self.G_bn10(h10), keep_prob=self.keep_prob)

        h10 = tf.nn.relu(h10)

        h10 = tf.concat([h10, h5_], axis=3)



        h11 = tf.nn.conv2d_transpose(h10, self.G_W11, output_shape=[batch_size, 8, 8, self.ch_G11], strides=[1, 2, 2, 1])  

        h11 = tf.nn.dropout(self.G_bn11(h11), keep_prob=self.keep_prob)

        h11 = tf.nn.relu(h11)

        h11 = tf.concat([h11, h4_], axis=3)



        h12 = tf.nn.conv2d_transpose(h11, self.G_W12, output_shape=[batch_size, 16, 16, self.ch_G12], strides=[1, 2, 2, 1])  
        
        h12 = self.G_bn12(h12)

        h12 = tf.nn.relu(h12)

        h12 = tf.concat([h12, h3_], axis=3)



        h13 = tf.nn.conv2d_transpose(h12, self.G_W13, output_shape=[batch_size, 32, 32, self.ch_G13], strides=[1, 2, 2, 1])  

        h13 = self.G_bn13(h13)

        h13 = tf.nn.relu(h13)

        h13 = tf.concat([h13, h2_], axis=3)



        h14 = tf.nn.conv2d_transpose(h13, self.G_W14, output_shape=[batch_size, 64, 64, self.ch_G14], strides=[1, 2, 2, 1])  

        h14 = self.G_bn14(h14)

        h14 = tf.nn.relu(h14)

        h14 = tf.concat([h14, h1_], axis=3)




        h16 = tf.nn.conv2d_transpose(h14, self.G_W16, output_shape=[batch_size, 128, 128, self.ch_G16], strides=[1, 2, 2, 1])  

        h16 = tf.nn.tanh(h16)




        generated_samples = self.sess.run(h16, feed_dict={input_img: input_image, con_input: cond_input})
        

        

        return generated_samples



    # Train Generator and return the loss

    def train_gen(self, input_img, target_img, con_input):

        

        _, loss_val_GAN, loss_val_L1 = self.sess.run([self.train_op_gen, self.G_loss_GAN, self.G_loss_L1], feed_dict={self.input_img: input_img, self.target_img: target_img, self.input_con: con_input})

      

        return loss_val_GAN, loss_val_L1



    # Train Discriminator and return the loss

    def train_discrim(self, input_img, target_img, input_condition):

        _, loss_val_D = self.sess.run([self.train_op_discrim, self.D_loss], feed_dict={self.input_img: input_img, self.target_img: target_img, self.input_con: input_condition})

        return loss_val_D
    

    
    # Build the Network

    def _build_model(self):

        self.input_con = tf.placeholder(tf.float32, [self.batch_size] + self.input_condition_shape)
        self.input_img = tf.placeholder(tf.float32, [self.batch_size] + self.input_image_shape)
        

        self.target_img = tf.placeholder(tf.float32, [self.batch_size] + self.output_image_shape)


        gen_img = self.generate(self.input_img, self.input_con)



        d_real = self.discriminate(self.input_img, self.target_img, self.input_con)

        d_fake = self.discriminate(self.input_img, gen_img, self.input_con)



        self.D_loss = tf.reduce_mean(tf.square(1-d_real) + tf.square(d_fake))
      

        self.G_loss_GAN = tf.reduce_mean(tf.square(1-d_fake))
        
        self.G_loss_L1 = tf.reduce_mean(tf.abs(self.target_img - gen_img))

      
        self.G_loss = self.G_loss_GAN + self.G_loss_L1 * self.l1_weight

       
        self.train_op_discrim = tf.train.AdamOptimizer(self.learning_rate, beta1=0.5).minimize(self.D_loss, var_list=self.discrim_params)

        self.train_op_gen = tf.train.AdamOptimizer(self.learning_rate, beta1=0.5).minimize(self.G_loss, var_list=self.gen_params)



In [3]:
#

if not os.path.exists(result_dir):
    os.makedirs(result_dir)

tf.reset_default_graph()
sess = tf.Session()
model = Pix2Pix(sess, batch_size)


saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state(ckpt_dir)

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)

else:
    sys.exit("There is no trained model")

print('Generating...')

# test data -> dataset 
test_input_path = []
test_output_path = []
test_condition_path = []

features_list = os.listdir(test_input_dir)
for feature_name in features_list :
    x = '%s/%s' %(test_input_dir, feature_name)
    test_input_path.append(x)
    
labels_list = os.listdir(test_output_dir)
for label_name in labels_list :
    x = '%s/%s' %(test_output_dir, label_name)
    test_output_path.append(x)
    
condition_list = os.listdir(test_condition_dir)
for condition_name in condition_list :
    x = '%s/%s' %(test_condition_dir, condition_name)
    test_condition_path.append(x)

def read_path(inputpath, outputpath, conditionpath):

    x = np.load(inputpath.decode(), allow_pickle = True) 
    y = np.load(outputpath.decode(), allow_pickle = True)
    z = np.load(conditionpath.decode(), allow_pickle = True)
    return x.astype(np.float64), y.astype(np.float64), z.astype(np.float64)
    
features, labels, conditions = (test_input_path, test_output_path, test_condition_path)
dataset = tf.data.Dataset.from_tensor_slices((features, labels, conditions))
dataset = dataset.map(lambda features, labels, conditions: tuple(tf.py_func(read_path, [features,labels, conditions], [tf.float64, tf.float64, tf.float64])))
dataset = dataset.batch(batch_size=batch_size).repeat(count=total_epoch).shuffle(buffer_size=shuffle_buffer_size)


iter = dataset.make_one_shot_iterator()
feature, label, condition = iter.get_next()


print('2Generating...')

for i in range(len(test_input_path)) :
    t_xs, t_ys, t_zs = sess.run([feature, label, condition])
    t_xs = np.reshape(t_xs,[1,128,128,1])
    t_zs = np.reshape(t_zs,[1,1,1,condition_shape])
    t_ys = np.reshape(t_ys,[1,128,128,1])
    generated_samples = model.sample_generator(t_xs, t_zs, batch_size=batch_size)
    
    real_initial = t_xs[0,:,:,0]
    real_final = t_ys[0,:,:,0]
    generated_samples = generated_samples[0,:,:,0]
    print('3Generating...')
    
    resize_real_initial = np.zeros(shape=(real_width,real_height))
    resize_real_final = np.zeros(shape=(real_width,real_height))
    resize_generated_samples = np.zeros(shape=(real_width,real_height))

    for W in range(real_width):
        for H in range(real_height):
            new_width = int( W * real_initial.shape[0] / real_width )
            new_height = int( H * real_initial.shape[1] / real_height )
            resize_real_initial[W,H] = real_initial[new_width,new_height]
            resize_real_final[W,H] = real_final[new_width,new_height]
            resize_generated_samples[W,H] = generated_samples[new_width,new_height]
            
    output3d_real_initial = np.zeros((real_width,real_depth,real_height))
    output3d_real_final = np.zeros((real_width,real_depth,real_height))
    output3d_generated_samples = np.zeros((real_width,real_depth,real_height))
    for q in range(real_width) :
        for r in range(real_height) :
            if bool(resize_real_initial[q,r]) :
                output3d_real_initial[q,0:int(resize_real_initial[q,r]*real_depth),r] = 1 
            if bool(resize_real_final[q,r]) :
                output3d_real_final[q,0:int(resize_real_final[q,r]*real_depth),r] = 1 
            if bool(resize_real_initial[q,r]) :
                output3d_generated_samples[q,0:int(resize_generated_samples[q,r]*real_depth),r] = 1 

 
    np.save(result_dir+'/generated{}.npy'.format(i),output3d_generated_samples)

print('finished!!')

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./output/checkpoint/5\model_epoch005
Generating...
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differ